<a href="https://colab.research.google.com/github/galhev/Featuretools/blob/master/featuretools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing with Featuretools & Sklearn-pandas

In [2]:
# Install package
pip install featuretools

In [3]:
# Install package
pip install sklearn-pandas

In [0]:
# Visualization:
# https://www.kaggle.com/rounakbanik/the-story-of-film

# Download dataset:
# https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings.csv


In [0]:
import pandas as pd
from sklearn_pandas import DataFrameMapper, gen_features,  CategoricalImputer
import sklearn.preprocessing
from datetime import date
import numpy as np

In [5]:
# save model image file to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
!ls -la /content/gdrive/My\ Drive/Colab\ Notebooks/featuretools

total 729065
-rw------- 1 root root    127307 Sep 16 13:52 featuretools.ipynb
-rw------- 1 root root  34445126 Sep 11 11:37 movies_metadata.csv
-rw------- 1 root root 709550327 Sep 11 11:38 ratings.csv
-rw------- 1 root root   2438266 Sep 15 18:27 ratings_small.csv


In [7]:
movies = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/featuretools/movies_metadata.csv')
ratings = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/featuretools/ratings.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [0]:
movies.rename(columns={'id': 'movieId'}, inplace=True)

In [0]:
movies['movieId'] = movies['movieId'].apply(lambda x: x if x.isdigit() else 999999999)

In [0]:
movies['budget'] = movies['budget'].apply(lambda x: x if x.isdigit() else 0)

In [0]:
movies['movieId'] = movies['movieId'].astype('int64')

In [13]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
adult                    45466 non-null object
belongs_to_collection    4494 non-null object
budget                   45466 non-null object
genres                   45466 non-null object
homepage                 7782 non-null object
movieId                  45466 non-null int64
imdb_id                  45449 non-null object
original_language        45455 non-null object
original_title           45466 non-null object
overview                 44512 non-null object
popularity               45461 non-null object
poster_path              45080 non-null object
production_companies     45463 non-null object
production_countries     45463 non-null object
release_date             45379 non-null object
revenue                  45460 non-null float64
runtime                  45203 non-null float64
spoken_languages         45460 non-null object
status                   45379 non-null object

In [14]:
movies.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
movieId                      0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [0]:
movies = movies.drop(['overview','homepage','original_title','imdb_id', 'belongs_to_collection', 'genres','poster_path', 'production_companies','production_countries','spoken_languages', 'tagline'], axis=1)

In [0]:
# Set up transformations for numerical and categorical columns
# col_categorical = list( movies.select_dtypes(exclude=[np.int64]) )

col_cat_list = list( movies.select_dtypes(exclude=np.number) )
col_num_list = list( movies.select_dtypes(include=np.number) )
col_none = ['movieId']

num_cols = ['budget', 'popularity']
[ col_cat_list.remove(x) for x in num_cols ]
[ col_num_list.append(x) for x in num_cols ]
col_cat_list.remove('release_date')
col_num_list.remove('movieId')

movies[col_cat_list] = movies[col_cat_list].astype('category')
# movies[col_num_list] = movies[col_num_list].astype('float64', errors='coerce')
movies['budget'] = pd.to_numeric(movies['budget'],errors="coerce")
movies['popularity'] = pd.to_numeric(movies['popularity'],errors="coerce")

# movies[col_num_list] = [ pd.to_numeric(movies[x], errors='coerce') for x in col_num_list ]

# Convert to list of lists
col_categorical = [ [x] for x in col_cat_list ]
col_numerical   = [ [x] for x in col_num_list ]
col_date = ['release_date']


In [17]:
movies.head()

,adult,budget,movieId,original_language,popularity,release_date,revenue,runtime,status,title,video,vote_average,vote_count
0,False,30000000,862,en,21.946943,1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0
1,False,65000000,8844,en,17.015539,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0
2,False,0,15602,en,11.712900,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0
3,False,16000000,31357,en,3.859495,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0
4,False,0,11862,en,8.387519,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0


In [0]:
movies['release_date']=pd.to_datetime(movies['release_date'], errors="coerce")

In [19]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 13 columns):
adult                45466 non-null category
budget               45466 non-null int64
movieId              45466 non-null int64
original_language    45455 non-null category
popularity           45460 non-null float64
release_date         45376 non-null datetime64[ns]
revenue              45460 non-null float64
runtime              45203 non-null float64
status               45379 non-null category
title                45460 non-null category
video                45460 non-null category
vote_average         45460 non-null float64
vote_count           45460 non-null float64
dtypes: category(5), datetime64[ns](1), float64(5), int64(2)
memory usage: 4.7 MB


In [0]:
#movies['release_date'].fillna('1900-01-01', inplace=True)

In [0]:
from sklearn.base import TransformerMixin

class DateEncoder(TransformerMixin):
  def fit(self, X, y=None):
    return self

  def transform(self, X):
    X = pd.to_datetime(movies['release_date'])
    dt = X.dt
    return pd.concat([dt.year, dt.month, dt.day],  axis=1)

In [0]:
classes_categorical = [ CategoricalImputer, sklearn.preprocessing.LabelEncoder]
classes_numerical = [ {'class':sklearn.preprocessing.Imputer, 'strategy' : 'median'}, sklearn.preprocessing.StandardScaler]
classes_dates = [DateEncoder]
classes_none = [None]

In [23]:
feature_def = gen_features(
    columns = col_categorical
    , classes = classes_categorical
)


feature_def_numerical = gen_features(
    columns = col_numerical
    , classes = classes_numerical
)


feature_def_date = gen_features(
    columns = col_date
    , classes = classes_dates
)


feature_def_none = gen_features(
    columns = col_none
    , classes = classes_none
)

feature_def.extend(feature_def_date)
feature_def.extend(feature_def_numerical)
feature_def.extend(feature_def_none)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was 

In [24]:
feature_def

[(['adult'],
  [CategoricalImputer(copy=True, fill_value='?', missing_values='NaN',
                      strategy='most_frequent'), LabelEncoder()]),
 (['original_language'],
  [CategoricalImputer(copy=True, fill_value='?', missing_values='NaN',
                      strategy='most_frequent'), LabelEncoder()]),
 (['status'],
  [CategoricalImputer(copy=True, fill_value='?', missing_values='NaN',
                      strategy='most_frequent'), LabelEncoder()]),
 (['title'],
  [CategoricalImputer(copy=True, fill_value='?', missing_values='NaN',
                      strategy='most_frequent'), LabelEncoder()]),
 (['video'],
  [CategoricalImputer(copy=True, fill_value='?', missing_values='NaN',
                      strategy='most_frequent'), LabelEncoder()]),
 ('release_date', [<__main__.DateEncoder at 0x7ff81b9784a8>]),
 (['revenue'],
  [Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0),
   StandardScaler(copy=True, with_mean=True, with_std=True)]),
 (['runt

In [25]:
mapper = DataFrameMapper(feature_def , df_out = True)
new_df_movies = mapper.fit_transform(movies)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [26]:
movies.head()

,adult,budget,movieId,original_language,popularity,release_date,revenue,runtime,status,title,video,vote_average,vote_count
0,False,30000000,862,en,21.946943,1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0
1,False,65000000,8844,en,17.015539,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0
2,False,0,15602,en,11.712900,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0
3,False,16000000,31357,en,3.859495,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0
4,False,0,11862,en,8.387519,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0


In [0]:
new_df_movies.rename(columns={'release_date_0': 'year', 'release_date_1': 'month', 'release_date_2':'day'}, inplace=True)

In [28]:
new_df_movies.head()

,adult,original_language,status,title,video,year,month,day,revenue,runtime,vote_average,vote_count,budget,popularity,movieId
0,3,20,4,39018,0,1995.0,10.0,30.0,5.632841,-0.342939,1.081946,10.798693,1.479373,3.168317,862
1,3,20,4,16806,0,1995.0,12.0,15.0,3.911075,0.257643,0.666161,4.688049,3.488165,2.347099,8844
2,3,20,4,13371,0,1995.0,12.0,22.0,-0.174232,0.179307,0.458269,-0.036404,-0.242450,1.464059,15602
3,3,20,4,40428,0,1995.0,12.0,22.0,1.091980,0.858225,0.250376,-0.154464,0.675855,0.156246,31357
4,3,20,4,11199,0,1995.0,2.0,10.0,1.016224,0.309868,0.042484,0.128474,-0.242450,0.910290,11862


In [0]:
new_df_movies[col_cat_list] = new_df_movies[col_cat_list].astype('category')

In [30]:
new_df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 15 columns):
adult                45466 non-null category
original_language    45466 non-null category
status               45466 non-null category
title                45466 non-null category
video                45466 non-null category
year                 45376 non-null float64
month                45376 non-null float64
day                  45376 non-null float64
revenue              45466 non-null float64
runtime              45466 non-null float64
vote_average         45466 non-null float64
vote_count           45466 non-null float64
budget               45466 non-null float64
popularity           45466 non-null float64
movieId              45466 non-null int64
dtypes: category(5), float64(9), int64(1)
memory usage: 5.4 MB


In [31]:
import featuretools as ft

ft.list_primitives()

,name,type,description
0,num_true,aggregation,Finds the number of 'True' values in a boolean.
1,count,aggregation,Counts the number of non null values.
2,mean,aggregation,Computes the average value of a numeric feature.
3,n_most_common,aggregation,Finds the N most common elements in a categori...
4,time_since_last,aggregation,Time since last related instance.
5,avg_time_between,aggregation,Computes the average time between consecutive ...
6,trend,aggregation,Calculates the slope of the linear trend of va...
7,median,aggregation,Finds the median value of any feature with wel...
8,mode,aggregation,Finds the most common element in a categorical...
9,min,aggregation,Finds the minimum non-null value of a numeric ...


In [0]:
new_df_movies = new_df_movies.drop_duplicates('movieId')

In [0]:
col_cat_list.append('year')
col_cat_list.append('month')
col_cat_list.append('day')

In [34]:
# initialize entityset
es = ft.EntitySet(id = 'movies_entitySet')
variable_types = {cat:ft.variable_types.Categorical for cat in col_cat_list}

es.entity_from_dataframe(entity_id = 'movies_entity_id', dataframe = new_df_movies,  make_index = False, index = 'movieId', variable_types = variable_types)


Entityset: movies_entitySet
  Entities:
    movies_entity_id [Rows: 45434, Columns: 15]
  Relationships:
    No relationships

In [35]:
es['movies_entity_id']

Entity: movies_entity_id
  Variables:
    movieId (dtype: index)
    revenue (dtype: numeric)
    runtime (dtype: numeric)
    vote_average (dtype: numeric)
    vote_count (dtype: numeric)
    budget (dtype: numeric)
    popularity (dtype: numeric)
    adult (dtype: categorical)
    original_language (dtype: categorical)
    status (dtype: categorical)
    title (dtype: categorical)
    video (dtype: categorical)
    year (dtype: categorical)
    month (dtype: categorical)
    day (dtype: categorical)
  Shape:
    (Rows: 45434, Columns: 15)

In [36]:
es['movies_entity_id'].df

,movieId,revenue,runtime,vote_average,vote_count,budget,popularity,adult,original_language,status,title,video,year,month,day
2,2,-0.174232,-0.656286,0.770107,-0.134109,-0.242450,0.156412,3,26,4,3036,0,1988.0,10.0,21.0
3,3,-0.174232,-0.473500,0.770107,-0.152429,-0.242450,-0.104768,3,26,4,27311,0,1986.0,10.0,16.0
5,5,-0.107386,0.100970,0.458269,0.873476,-0.012873,1.016712,3,20,4,11966,0,1995.0,12.0,9.0
6,6,0.014443,0.414317,0.406296,-0.062865,-0.242450,0.435876,3,20,4,16770,0,1993.0,10.0,15.0
11,11,11.879695,0.701552,1.289838,13.573113,0.388885,6.532646,3,20,4,28841,0,1977.0,5.0,25.0
12,12,14.443726,0.153194,1.029973,12.583848,5.152594,3.759634,3,20,4,11441,0,2003.0,5.0,30.0
13,13,10.364748,1.249909,1.341811,16.359746,2.914225,7.558043,3,20,4,11913,0,1994.0,7.0,6.0
14,14,5.364566,0.727664,1.185892,6.774461,0.618461,2.965091,3,20,4,2384,0,1999.0,9.0,15.0
15,15,0.186698,0.649327,1.237865,2.308520,-0.194254,2.146662,3,20,4,7036,0,1941.0,4.0,30.0
16,16,0.448083,1.197684,1.081946,0.574254,0.492194,1.292853,3,20,4,8213,0,2000.0,5.0,17.0


In [37]:
movies_new_features, feature_defs = ft.dfs(entityset = es, 
                                    target_entity = 'movies_entity_id',
                                    trans_primitives = ['add', 'multiply'])

movies_new_features.head()

,revenue,runtime,vote_average,vote_count,budget,popularity,adult,original_language,status,title,video,year,month,day,revenue + runtime,vote_average + vote_count,revenue + vote_average,budget + vote_average,runtime + vote_average,popularity + vote_average,budget + popularity,revenue + vote_count,runtime + vote_count,popularity + runtime,popularity + revenue,budget + revenue,popularity + vote_count,budget + vote_count,budget + runtime,revenue * vote_average,budget * vote_average,budget * revenue,budget * runtime,popularity * vote_average,popularity * vote_count,runtime * vote_average,revenue * vote_count,revenue * runtime,vote_average * vote_count,budget * popularity,...,budget + popularity * revenue,budget + popularity * runtime,popularity + vote_count * vote_count,popularity + runtime * revenue + vote_count,popularity + vote_average * vote_average,revenue + vote_average * vote_count,budget + vote_count * popularity + runtime,budget + vote_count * popularity + vote_average,popularity + vote_count * runtime + vote_count,vote_average + vote_count * vote_count,budget + popularity * popularity + runtime,budget + vote_average * revenue,budget + runtime * revenue + vote_count,revenue + vote_average * revenue + vote_count,budget + popularity * vote_count,revenue + runtime * vote_count,budget + popularity * popularity + vote_count,revenue + vote_count * vote_count,budget + runtime * runtime + vote_count,budget * budget + revenue,budget + revenue * runtime + vote_count,budget + vote_count * revenue,popularity + revenue * runtime + vote_count,revenue + runtime * revenue + vote_average,revenue + vote_count * runtime + vote_count,budget + runtime * vote_count,budget + popularity * revenue + vote_average,budget + revenue * revenue + vote_count,popularity + vote_average * vote_average + vote_count,budget + popularity * popularity + vote_average,popularity + revenue * runtime,budget + vote_count * revenue + runtime,budget + runtime * vote_average,revenue * vote_average + vote_count,budget + revenue * runtime,budget + vote_count * popularity + vote_count,budget + runtime * popularity,budget + popularity * budget + vote_average,budget + vote_average * vote_average,budget + popularity * revenue + vote_count
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,-0.174232,-0.656286,0.770107,-0.134109,-0.242450,0.156412,3,26,4,3036,0,1988.0,10.0,21.0,-0.830517,0.635998,0.595876,0.527657,0.113822,0.926519,-0.086038,-0.308340,-0.790394,-0.499874,-0.017820,-0.416681,0.022303,-0.376559,-0.898735,-0.134177,-0.186712,0.042242,0.159116,0.120454,-0.020976,-0.505410,0.023366,0.114346,-0.103278,-0.037922,...,0.014991,0.056466,-0.002991,0.154131,0.713519,-0.079912,0.188232,-0.348889,-0.017628,-0.085293,0.043008,-0.091935,0.277116,-0.183733,0.011538,0.111380,-0.001919,0.041351,0.710355,0.101024,0.329343,0.065608,0.014085,-0.494885,0.243711,0.120528,-0.051268,0.128480,0.589264,-0.079716,0.011695,0.312738,-0.692123,-0.110811,0.273462,-0.008398,-0.140573,-0.045399,0.406353,0.026529
3,-0.174232,-0.473500,0.770107,-0.152429,-0.242450,-0.104768,3,26,4,27311,0,1986.0,10.0,16.0,-0.647731,0.617679,0.595876,0.527657,0.296607,0.665339,-0.347218,-0.326660,-0.625928,-0.578268,-0.279000,-0.416681,-0.257197,-0.394878,-0.715950,-0.134177,-0.186712,0.042242,0.114800,-0.080683,0.015970,-0.364646,0.026558,0.082499,-0.117386,0.025401,...,0.060496,0.164408,0.039204,0.188897,0.512382,-0.090828,0.228346,-0.262728,0.160987,-0.094152,0.200785,-0.091935,0.233872,-0.194649,0.052926,0.098733,0.089303,0.049792,0.448133,0.101024,0.260813,0.068800,0.174634,-0.385967,0.204466,0.109131,-0.206899,0.136113,0.410966,-0.231018,0.132106,0.255775,-0.551358,-0.107619,0.197299,0.101561,0.075009,-0.183212,0.406353,0.113422
5,-0.107386,0.100970,0.458269,0.873476,-0.012873,1.016712,3,20,4,11966,0,1995.0,12.0,9.0,-0.006416,1.331744,0.350883,0.445395,0.559238,1.474981,1.003839,0.766090,0.974446,1.117682,0.909326,-0.120260,1.890188,0.860602,0.088096,-0.049212,-0.005900,

In [38]:
feature_defs

[<Feature: revenue>,
 <Feature: runtime>,
 <Feature: vote_average>,
 <Feature: vote_count>,
 <Feature: budget>,
 <Feature: popularity>,
 <Feature: adult>,
 <Feature: original_language>,
 <Feature: status>,
 <Feature: title>,
 <Feature: video>,
 <Feature: year>,
 <Feature: month>,
 <Feature: day>,
 <Feature: revenue + runtime>,
 <Feature: vote_average + vote_count>,
 <Feature: revenue + vote_average>,
 <Feature: budget + vote_average>,
 <Feature: runtime + vote_average>,
 <Feature: popularity + vote_average>,
 <Feature: budget + popularity>,
 <Feature: revenue + vote_count>,
 <Feature: runtime + vote_count>,
 <Feature: popularity + runtime>,
 <Feature: popularity + revenue>,
 <Feature: budget + revenue>,
 <Feature: popularity + vote_count>,
 <Feature: budget + vote_count>,
 <Feature: budget + runtime>,
 <Feature: revenue * vote_average>,
 <Feature: budget * vote_average>,
 <Feature: budget * revenue>,
 <Feature: budget * runtime>,
 <Feature: popularity * vote_average>,
 <Feature: popula

In [39]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [40]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
userId       int64
movieId      int64
rating       float64
timestamp    int64
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


In [53]:
es.entity_from_dataframe(entity_id = 'ratings_entity_id', dataframe = ratings, 
                         make_index = True, index = 'rating_id',
                         variable_types = {'userId': ft.variable_types.Categorical, 'timestamp': ft.variable_types.Categorical})

Entityset: movies_entitySet
  Entities:
    movies_entity_id [Rows: 45434, Columns: 15]
    ratings_entity_id [Rows: 26024289, Columns: 5]
  Relationships:
    ratings_entity_id.movieId -> movies_entity_id.movieId

In [54]:
# add relationships
movies_ratings_relation = ft.Relationship(es['movies_entity_id']['movieId'],
                                          es['ratings_entity_id']['movieId'])

es = es.add_relationship(movies_ratings_relation)

2019-09-16 14:03:32,518 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: ratings_entity_id.movieId -> movies_entity_id.movieId>


In [55]:
es

Entityset: movies_entitySet
  Entities:
    movies_entity_id [Rows: 45434, Columns: 15]
    ratings_entity_id [Rows: 26024289, Columns: 5]
  Relationships:
    ratings_entity_id.movieId -> movies_entity_id.movieId

In [56]:
es['ratings_entity_id']

Entity: ratings_entity_id
  Variables:
    rating_id (dtype: index)
    movieId (dtype: ordinal)
    rating (dtype: numeric)
    userId (dtype: categorical)
    timestamp (dtype: categorical)
  Shape:
    (Rows: 26024289, Columns: 5)

In [0]:
df, features = ft.dfs(entityset = es, target_entity = 'movies_entity_id', max_depth = 2)

# df, features = ft.dfs(entityset = es, target_entity = 'movies_entity_id', agg_primitives = ['mean'])

In [58]:
df.head(5)

,revenue,runtime,vote_average,vote_count,budget,popularity,adult,original_language,status,title,video,year,month,day,SUM(ratings_entity_id.rating),STD(ratings_entity_id.rating),MAX(ratings_entity_id.rating),SKEW(ratings_entity_id.rating),MIN(ratings_entity_id.rating),MEAN(ratings_entity_id.rating),COUNT(ratings_entity_id),NUM_UNIQUE(ratings_entity_id.userId),NUM_UNIQUE(ratings_entity_id.timestamp),MODE(ratings_entity_id.userId),MODE(ratings_entity_id.timestamp)
movieId,,,,,,,,,,,,,,,,,,,,,,,,,
2,-0.174232,-0.656286,0.770107,-0.134109,-0.242450,0.156412,3,26,4,3036,0,1988.0,10.0,21.0,84355.0,0.958437,5.0,-0.318964,0.5,3.236953,26060.0,26060.0,26058.0,24.0,849122404.0
3,-0.174232,-0.473500,0.770107,-0.152429,-0.242450,-0.104768,3,26,4,27311,0,1986.0,10.0,16.0,49211.5,1.013307,5.0,-0.241727,0.5,3.175550,15497.0,15497.0,15495.0,46.0,860156485.0
5,-0.107386,0.100970,0.458269,0.873476,-0.012873,1.016712,3,20,4,11966,0,1995.0,12.0,9.0,46988.0,0.993108,5.0,-0.243825,0.5,3.079565,15258.0,15258.0,15257.0,2.0,848992200.0
6,0.014443,0.414317,0.406296,-0.062865,-0.242450,0.435876,3,20,4,16770,0,1993.0,10.0,15.0,107166.0,0.883211,5.0,-0.589678,0.5,3.841764,27895.0,27895.0,27893.0,15.0,844965565.0
11,11.879695,0.701552,1.289838,13.573113,0.388885,6.532646,3,20,4,28841,0,1977.0,5.0,25.0,71290.0,0.906568,5.0,-0.502945,0.5,3.660591,19475.0,19475.0,19470.0,6.0,822873600.0


In [59]:
features

[<Feature: revenue>,
 <Feature: runtime>,
 <Feature: vote_average>,
 <Feature: vote_count>,
 <Feature: budget>,
 <Feature: popularity>,
 <Feature: adult>,
 <Feature: original_language>,
 <Feature: status>,
 <Feature: title>,
 <Feature: video>,
 <Feature: year>,
 <Feature: month>,
 <Feature: day>,
 <Feature: SUM(ratings_entity_id.rating)>,
 <Feature: STD(ratings_entity_id.rating)>,
 <Feature: MAX(ratings_entity_id.rating)>,
 <Feature: SKEW(ratings_entity_id.rating)>,
 <Feature: MIN(ratings_entity_id.rating)>,
 <Feature: MEAN(ratings_entity_id.rating)>,
 <Feature: COUNT(ratings_entity_id)>,
 <Feature: NUM_UNIQUE(ratings_entity_id.userId)>,
 <Feature: NUM_UNIQUE(ratings_entity_id.timestamp)>,
 <Feature: MODE(ratings_entity_id.userId)>,
 <Feature: MODE(ratings_entity_id.timestamp)>]